**Using another model for Sentiment Analysis:** pysentimiento

In [1]:
import pandas as pd

In [2]:
!pip install pysentimiento

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.wh

In [3]:
from pysentimiento import create_analyzer

In [18]:
spanish_analyzer = create_analyzer(task="sentiment", lang="es")
english_analyzer = create_analyzer(task="sentiment", lang="en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [19]:

def apply_sentiment_analysis(data, spanish_column, english_column):
    # Define a function to apply sentiment analysis and return the probabilities
    def analyze_sentiment(text, analyzer):
        if pd.isna(text):
            # Return NaN scores if the text is NaN
            return {'NEU': None, 'NEG': None, 'POS': None}
        else:
            # Ensure the text is a string
            text = str(text)
            result = analyzer.predict(text)
            return result.probas

    # Apply sentiment analysis to the Spanish column and create new columns for each sentiment score
    data[[f'{spanish_column}_NEU', f'{spanish_column}_NEG', f'{spanish_column}_POS']] = data[spanish_column].apply(
        lambda text: pd.Series(analyze_sentiment(text, spanish_analyzer))
    )

    # Apply sentiment analysis to the English column and create new columns for each sentiment score
    data[[f'{english_column}_NEU', f'{english_column}_NEG', f'{english_column}_POS']] = data[english_column].apply(
        lambda text: pd.Series(analyze_sentiment(text, english_analyzer))
    )

    return data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read the CSV file
data = pd.read_csv('/content/drive/MyDrive/522Project/combined_transcripts_Eng_to_Spa.csv')
data.columns = ['English', 'Spanish']

In [ ]:
#Apply function in compiled transcripts originally English and translate to Spanish
data = apply_sentiment_analysis(data, 'Spanish', 'English')

# Display the updated DataFrame
data.head()


,English,Spanish,Spanish_NEU,Spanish_NEG,Spanish_POS,English_NEU,English_NEG,English_POS
0,It’s 2009 Alyssa Elliot is sitting at her desk...,"Es 2009, Alyssa Elliot está sentada en su escr...",0.950366,0.045812,0.003822,0.767613,0.226484,0.005903
1,You say things like I couldn’t have foreseen t...,Dices cosas como si no hubiera podido prever q...,0.799058,0.180481,0.020460,0.837934,0.152790,0.009275
2,"This is Last Day, a show about the moments tha...","Este es Last Day, un programa sobre los moment...",0.024352,0.552673,0.422975,0.005896,0.930141,0.063963
3,"Alyssa grew up in Boise, Idaho with her mom, d...","Alyssa creció en Boise, Idaho con su mamá, su ...",0.027817,0.615897,0.356286,0.003801,0.940731,0.055467
4,That is with an S not a Z biggest pet peeve.,"Eso es con una S, no con una Z, lo que más le ...",0.744763,0.236864,0.018373,0.946207,0.049390,0.004403


In [14]:
data2 = pd.read_csv('/content/drive/MyDrive/522Project/combined_transcripts_Spa_to_Eng.csv')
data2.columns = ['Spanish', 'English']
data2.shape

(1238, 2)

In [13]:
#Apply function in compiled transcripts originally Spanish and translate to English
data2 = apply_sentiment_analysis(data2, 'Spanish', 'English')

# Display the updated DataFrame
data2.shape

NameError: name 'spanish_analyzer' is not defined

In [ ]:
idiom1 = pd.read_csv('/content/drive/MyDrive/522Project/idioms/Spanish_idioms.csv')
idiom2 = pd.read_csv('/content/drive/MyDrive/522Project/idioms/Spanish_idioms_1.csv')
idiom3 = pd.read_csv('/content/drive/MyDrive/522Project/idioms/Spanish_idioms_2.csv')
idiom4 = pd.read_csv('/content/drive/MyDrive/522Project/idioms/Spanish_idioms_3.csv')

idiom = pd.concat([idiom1, idiom2, idiom3, idiom4]).reset_index(drop=True)
idiom.drop(columns=['literal_translation', 'Example'], axis=1, inplace=True)
idiom.head()

,Idiom,Meaning
0,"A buen hambre, no hay pan duro.",Beggars can’t be choosers.
1,estar como unasopa,to be soaked to the bone
2,estar como un fideo,to be thin as a rail
3,estar como un queso,to be hot (sexually attractive)
4,no importar un pepino,to not care at all


In [ ]:
#Apply function in compiled idioms in Spanish and Meaning in English
data3 = apply_sentiment_analysis(idiom, 'Idiom', 'Meaning')

# Display the updated DataFrame
data3.head()

,Idiom,Meaning,Idiom_NEU,Idiom_NEG,Idiom_POS,Meaning_NEU,Meaning_NEG,Meaning_POS
0,"A buen hambre, no hay pan duro.",Beggars can’t be choosers.,0.274419,0.552515,0.173066,0.889243,0.104812,0.005945
1,estar como unasopa,to be soaked to the bone,0.254890,0.658922,0.086187,0.821744,0.171732,0.006524
2,estar como un fideo,to be thin as a rail,0.665327,0.290289,0.044384,0.127502,0.852656,0.019842
3,estar como un queso,to be hot (sexually attractive),0.739185,0.228126,0.032689,0.002461,0.077930,0.919609
4,no importar un pepino,to not care at all,0.690299,0.237997,0.071704,0.884299,0.109150,0.006551


In [ ]:
# Save the dataframes with predictions to a new CSV
data3.to_csv('/content/drive/MyDrive/522Project/py_sent_idioms.csv', index=False)
data2.to_csv('/content/drive/MyDrive/522Project/py_sent_Spa_to_Eng.csv', index=False)
data.to_csv('/content/drive/MyDrive/522Project/py_sent_Eng_to_Spa.csv', index=False)

In [20]:
gpt1_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Amponas_gpt.csv', encoding='ISO-8859-1')
gpt2_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Barberias_gpt.csv')
gpt3_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Damitas_gpt.csv')
gpt4_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Farid_gpt.csv')
gpt5_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Fobias_gpt.csv')
gpt6_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Ovnis_gpt.csv')
gpt7_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Subio_gpt.csv')
gpt8_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Techos_gpt.csv')
gpt9_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Triangulo_gpt.csv')


gpt_spa_to_eng = pd.concat([gpt1_spa_to_eng, gpt2_spa_to_eng, gpt3_spa_to_eng, gpt4_spa_to_eng, gpt5_spa_to_eng, gpt6_spa_to_eng, gpt7_spa_to_eng, gpt8_spa_to_eng, gpt9_spa_to_eng]).reset_index(drop=True)
gpt_spa_to_eng.head()

,Original,gpt4
0,"¿Cómo estás, soy turbulent? Y yo la burrita bu...","How are you, am I turbulent? And I the little ..."
1,Son largas y aparte esta esta mención es. Para...,The seasons are long and besides this mention ...
2,Y han pasado personas muy importantes por esta...,And very important people have passed through ...
3,Ciclo formal y a partir de aquí el podcast del...,A formal cycle and from here the podcast of th...
4,Pero hay que aclarar que obviamente seguiremos...,But we must clarify that obviously we will con...


In [22]:
#Apply function in compiled transcripts originally English and translate to Spanish
gpt_spa = apply_sentiment_analysis(gpt_spa_to_eng, 'Original', 'gpt4')

# Display the updated DataFrame
gpt_spa.head()

,Original,gpt4,Original_NEU,Original_NEG,Original_POS,gpt4_NEU,gpt4_NEG,gpt4_POS
0,"¿Cómo estás, soy turbulent? Y yo la burrita bu...","How are you, am I turbulent? And I the little ...",0.612038,0.375407,0.012556,0.150018,0.641487,0.208495
1,Son largas y aparte esta esta mención es. Para...,The seasons are long and besides this mention ...,0.310165,0.511624,0.178211,0.001756,0.019263,0.978982
2,Y han pasado personas muy importantes por esta...,And very important people have passed through ...,0.808546,0.185225,0.006229,0.533023,0.459934,0.007042
3,Ciclo formal y a partir de aquí el podcast del...,A formal cycle and from here the podcast of th...,0.587000,0.402197,0.010803,0.299618,0.641958,0.058424
4,Pero hay que aclarar que obviamente seguiremos...,But we must clarify that obviously we will con...,0.004094,0.105816,0.890090,0.001452,0.203300,0.795249


In [24]:
# Save the dataframes with predictions to a new CSV
gpt_spa.to_csv('/content/drive/MyDrive/522Project/gpt_spa_to_eng.csv', index=False)

In [32]:
gpt1_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Flaps_gpt.csv', encoding='ISO-8859-1')
gpt2_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Highway_gpt.csv', encoding='ISO-8859-1')
gpt3_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Mariko_gpt.csv', encoding='ISO-8859-1')
gpt4_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Unfit_gpt.csv', encoding='ISO-8859-1')
gpt5_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Wax_gpt.csv', encoding='ISO-8859-1')
gpt6_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Alyssa_gpt.csv', encoding='ISO-8859-1')
gpt7_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Carrie_gpt.csv', encoding='ISO-8859-1')
gpt8_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Awkward_gpt.csv' , encoding='ISO-8859-1')


gpt_eng_to_spa = pd.concat([gpt1_eng_to_spa, gpt2_eng_to_spa, gpt3_eng_to_spa, gpt4_eng_to_spa, gpt5_eng_to_spa, gpt6_eng_to_spa, gpt7_eng_to_spa, gpt8_eng_to_spa]).reset_index(drop=True)
gpt_eng_to_spa.head()

,Original,gpt4
0,"1. Hi everybody, it's your old pal Sarah. Over...","1. Hola a todos, soy vuestra vieja amiga Sarah..."
1,2. So ridiculous. And I had to wear Spanx unde...,2. Tan ridículo. Y tuve que llevar Spanx debaj...
2,3. Not that there's anything wrong with that. ...,3. No es que haya algo malo en eso. Pero tenía...
3,4. And when I can't wear a tight skirt without...,4. Y cuando no puedo llevar una falda ajustada...
4,5. It burns like fire. So I was wearing these ...,5. Arde como fuego. Así que llevaba estos Span...


In [33]:
#Apply function in compiled transcripts originally English and translate to Spanish
gpt_eng = apply_sentiment_analysis(gpt_eng_to_spa, 'gpt4', 'Original')

# Display the updated DataFrame
gpt_eng.head()

,Original,gpt4,gpt4_NEU,gpt4_NEG,gpt4_POS,Original_NEU,Original_NEG,Original_POS
0,"1. Hi everybody, it's your old pal Sarah. Over...","1. Hola a todos, soy vuestra vieja amiga Sarah...",0.032806,0.674775,0.292419,0.018466,0.956572,0.024962
1,2. So ridiculous. And I had to wear Spanx unde...,2. Tan ridículo. Y tuve que llevar Spanx debaj...,0.933835,0.062783,0.003382,0.971826,0.025568,0.002605
2,3. Not that there's anything wrong with that. ...,3. No es que haya algo malo en eso. Pero tenía...,0.077322,0.679731,0.242947,0.255475,0.731315,0.013210
3,4. And when I can't wear a tight skirt without...,4. Y cuando no puedo llevar una falda ajustada...,0.892831,0.099999,0.007170,0.322705,0.657761,0.019533
4,5. It burns like fire. So I was wearing these ...,5. Arde como fuego. Así que llevaba estos Span...,0.781700,0.207685,0.010615,0.524018,0.465672,0.010309


In [34]:
# Save the dataframes with predictions to a new CSV
gpt_eng.to_csv('/content/drive/MyDrive/522Project/gpt_eng_to_spa.csv', index=False)